## Install and Import

In [1]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 11.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import random
import csv
import json
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


## Data Loading

We load the training data and the document embedding (embedder mpnet-base-v2 without chunking).

In [13]:
# Load train.csv
train_df = pd.read_csv('/kaggle/input/dis-project-1-document-retrieval/train.csv')

# Load document embeddings
with open('/kaggle/input/dis-embedding/doc_embedding.pkl', 'rb') as f:
    doc_embeddings = pickle.load(f)



In [14]:
train_df = train_df.sample(frac=0.5)

## Query Embedding

We embed the query with the same multilingual embedder.

In [15]:
# Load the pre-trained SentenceTransformer model
embedder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Function to embed queries using the SentenceTransformer model
def embed_query(query):
    # Get embedding for the query
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    return query_embedding

# Embed all queries in the dataset
train_df['query_embedding'] = train_df['query'].apply(embed_query)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Function to get document embeddings based on document ID
def get_doc_embedding(doc_id):
    return torch.tensor(doc_embeddings.get(doc_id, np.zeros(768)), dtype=torch.float32)

# Get embeddings for positive documents
train_df['positive_doc_embedding'] = train_df['positive_docs'].apply(get_doc_embedding)

# Get embeddings for negative documents (assuming multiple negative doc IDs per query)
train_df['negative_doc_embeddings'] = train_df['negative_docs'].apply(lambda doc_ids: [get_doc_embedding(doc_id) for doc_id in doc_ids.split(',')])

NameError: name 'train_df' is not defined

In [5]:
def prepare_training_sample(row):
    # Move query embedding to the device
    query_emb = row['query_embedding'].to(device)
    
    # Move positive document embedding to the device
    pos_doc_emb = row['positive_doc_embedding'].to(device)
    
    # Create a positive sample by concatenating query and positive document embedding
    pos_sample = torch.cat((query_emb, pos_doc_emb), dim=-1)
    
    # Create negative samples by concatenating query and each negative document embedding
    neg_samples = []
    for neg_emb in row['negative_doc_embeddings']:
        neg_sample = torch.cat((query_emb, neg_emb.to(device)), dim=-1)  # Move each negative doc embedding to the device
        neg_samples.append(neg_sample)
    
    return pos_sample, neg_samples

# Apply the function to each row in the dataframe
train_samples = train_df.apply(prepare_training_sample, axis=1)


NameError: name 'train_df' is not defined

## Define Ranking Model

A listwise neural network ranking model.

In [3]:
class RankingModel(nn.Module):
    def __init__(self, input_dim):
        super(RankingModel, self).__init__()
        # Adding dropout for regularization
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(p=0.2)  # Dropout layer for regularization
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(64, 1)  # Output a single scalar score for each query-document pair

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)  # Return a single score (no activation for regression output)


In [4]:
def listwise_loss(pred_scores, true_scores):
    """
    Implements listwise cross-entropy loss for ListNet, where true labels are binary (one-hot).
    
    Args:
    - pred_scores: Predicted relevance scores for the documents (output from the neural network).
    - true_scores: Ground truth binary relevance scores for the documents.
    
    Returns:
    - loss: The listwise cross-entropy loss.
    """
    # Ensure pred_scores and true_scores have the same dimensions
    pred_scores = pred_scores.view(-1)  # Flatten predicted scores
    true_scores = true_scores.view(-1)  # Flatten true scores (binary list)
    
    # Apply softmax to predicted scores (to convert them into a probability distribution)
    pred_probs = F.softmax(pred_scores, dim=0)
    
    # Use true_scores directly as a probability distribution (binary one-hot)
    true_probs = F.softmax(true_scores, dim=0)  # Apply softmax to true scores
    
    # Compute cross-entropy between predicted and true distributions
    loss = -torch.sum(true_probs * torch.log(pred_probs + 1e-8))  # Add epsilon for numerical stability
    
    return loss


## Model Training

In [20]:
def train_ranking_model(train_samples, model, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()
        #index = 0
        
        for i, (pos_sample, neg_samples) in enumerate(train_samples):
            #index += 1
            pos_sample = pos_sample.to(device)
            neg_samples = [neg_sample.to(device) for neg_sample in neg_samples]

            all_samples = [pos_sample] + neg_samples
            true_scores = [1] + [0] * len(neg_samples)

            # Shuffle documents and scores
            combined = list(zip(all_samples, true_scores))
            random.shuffle(combined)
            all_samples, true_scores = zip(*combined)

            true_scores = torch.tensor(true_scores, dtype=torch.float32).to(device)
            
            # Predict scores for all documents (output should be a single score per sample)
            all_scores = torch.stack([model(sample.unsqueeze(0)) for sample in all_samples]).squeeze()

            # Ensure that all_scores and true_scores have the same shape
            assert all_scores.shape == true_scores.shape, f"Shape mismatch: {all_scores.shape} vs {true_scores.shape}"

            # Compute listwise loss
            loss = listwise_loss(all_scores, true_scores)
            #if index <= 50:
            #    print(all_scores, true_scores)
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_samples):.4f}")

# Instantiate model and optimizer
input_dim = train_samples.iloc[0][0].shape[-1]  # Assuming 768*2 (query + doc embeddings)
model = RankingModel(input_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_ranking_model(train_samples, model, optimizer, device)


Epoch [1/10], Loss: 3.0047
Epoch [2/10], Loss: 3.0041
Epoch [3/10], Loss: 3.0040
Epoch [4/10], Loss: 3.0040
Epoch [5/10], Loss: 3.0040
Epoch [6/10], Loss: 3.0040
Epoch [7/10], Loss: 3.0040
Epoch [8/10], Loss: 3.0039
Epoch [9/10], Loss: 3.0039
Epoch [10/10], Loss: 3.0039


In [21]:
# Save the trained model
model_save_path = '/kaggle/working/ranking_model_50.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /kaggle/working/ranking_model_50.pth


## Ranking model application

We had a pre-filtered process to get 100 candidate (documents) for each query in the test set. Among these candidates, we applied our trained ranking model to get the most relevant 10 as submission.

In [22]:
model.eval()  # Make sure the model is in evaluation mode

# Load the query and top 100 document embeddings file (from Step 1)
def load_top_100_candidates(file_name):
    """
    Load the CSV file containing query embeddings and top 100 document embeddings.
    """
    return pd.read_csv(file_name)

def rank_top_10_for_each_query(model, candidates_df, device):
    """
    Rank the top 100 candidates and select the top 10 for each query.
    
    Args:
    - model: The trained ranking model.
    - candidates_df: DataFrame containing query and document embeddings.
    - device: The computation device (GPU or CPU).
    
    Returns:
    - final_results: A dictionary containing query IDs and top 10 document IDs.
    """
    final_results = {}
    
    for idx, row in candidates_df.iterrows():
        query_id = row['query']  # Assuming "query" column represents the query ID
        query_emb = torch.tensor(json.loads(row['query_embedding']), dtype=torch.float32).to(device)
        
        # Convert the top 100 document embeddings (stored as JSON string in CSV) back to tensors
        doc_embeddings = json.loads(row['top_100_doc_embeddings'])  # Deserialize the JSON string
        doc_embeddings_tensor = torch.tensor(doc_embeddings, dtype=torch.float32).to(device)  # Convert to tensor

        # Check that the shape matches (100 documents, embedding_dim)
        assert doc_embeddings_tensor.shape == (100, query_emb.shape[0]), \
            f"Expected shape (100, {query_emb.shape[0]}), got {doc_embeddings_tensor.shape}"
        
        # Prepare input for the model (query + document concatenation)
        inputs = torch.cat([torch.cat((query_emb, doc_emb)).unsqueeze(0) for doc_emb in doc_embeddings_tensor], dim=0)
        
        # Predict scores for all 100 documents
        with torch.no_grad():
            scores = model(inputs).squeeze()  # Get the scores for the documents
        
        # Get the indices of the top 10 highest-scoring documents
        top_10_indices = torch.argsort(scores, descending=True)[:10].cpu().numpy()
        
        # Select the top 10 document IDs based on the indices
        top_10_doc_ids = np.array(json.loads(row['top_100_doc_ids']))[top_10_indices].tolist()
        
        # Store the result for this query
        final_results[query_id] = top_10_doc_ids

    return final_results


In [23]:

# Step 1: Load the top 100 candidates from the CSV file
candidates_df = load_top_100_candidates("/kaggle/input/faiss100/query_top_100_candidates.csv")

In [24]:
# Step 2: Rank the top 100 documents and select top 10 using the trained model
final_results = rank_top_10_for_each_query(model, candidates_df, device)


In [27]:
# Step 3: Store the final results in a CSV file with "id" and "docids" columns
with open('/kaggle/working/query_top_10_results.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["id", "docids"])
    writer.writeheader()  # Write the column names
    
    for query_id, doc_ids in final_results.items():
        # Convert the doc_ids list to JSON string
        docids_str = json.dumps(doc_ids).replace('"', "'")
        
        # Write the row with the query id and the doc_ids
        writer.writerow({
            "id": query_id,
            "docids": docids_str  # Save the list of docids with single quotes
        })

print("Top 10 document results stored in 'query_top_10_results.csv'")


Top 10 document results stored in 'query_top_10_results.csv'


In [7]:
# Load precomputed document embeddings (from the "doc_embedding.pkl" file)
def load_precomputed_embeddings(embedding_file):
    """
    Load the document embeddings from a .pkl file.
    Returns a dictionary where keys are doc IDs and values are embeddings.
    """
    return np.load(embedding_file, allow_pickle=True)  # Load the doc embeddings as a dictionary

# Load the trained ranking model from a .pth file
def load_model(model_path, device):
    """
    Load the ranking model from the specified path.
    """
    model = RankingModel(input_dim=1536).to(device)  # Assuming input_dim = 768 (query) + 768 (doc)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()  # Put the model in evaluation mode
    return model

# Load the query and top 100 document IDs (no embeddings) from the CSV file
def load_top_100_candidates(file_name):
    """
    Load the CSV file containing query embeddings and top 100 document IDs.
    """
    return pd.read_csv(file_name)

def rank_top_10_for_each_query(model, candidates_df, doc_embeddings, device):
    """
    Rank the top 100 candidates and select the top 10 for each query.
    
    Args:
    - model: The trained ranking model.
    - candidates_df: DataFrame containing query embeddings and top 100 document IDs.
    - doc_embeddings: Dictionary mapping doc IDs to their embeddings.
    - device: The computation device (GPU or CPU).
    
    Returns:
    - final_results: A dictionary containing generated query IDs (0-1999) and top 10 document IDs.
    """
    final_results = {}
    
    for idx, row in candidates_df.iterrows():
        query_id = idx  # Generate query IDs from 0 to 1999
        
        # Extract the query embedding
        query_emb = torch.tensor(json.loads(row['query_embedding']), dtype=torch.float32).to(device)
        
        # Fetch the top 100 document IDs
        top_100_doc_ids = json.loads(row['top_200_doc_ids'])  # Deserialize the JSON string containing doc IDs
        
        # Fetch embeddings for the top 100 documents using the document ID
        doc_embeddings_tensor = torch.tensor(
            [doc_embeddings[doc_id] for doc_id in top_100_doc_ids],  # Fetch each embedding from the dictionary
            dtype=torch.float32
        ).to(device)

        # Check that the shape matches (100 documents, embedding_dim)
        assert doc_embeddings_tensor.shape == (200, query_emb.shape[0]), \
            f"Expected shape (200, {query_emb.shape[0]}), got {doc_embeddings_tensor.shape}"
        
        # Prepare input for the model (query + document concatenation)
        inputs = torch.cat([torch.cat((query_emb, doc_emb)).unsqueeze(0) for doc_emb in doc_embeddings_tensor], dim=0)
        
        # Predict scores for all 100 documents
        with torch.no_grad():
            scores = model(inputs).squeeze()  # Get the scores for the documents
        
        # Get the indices of the top 10 highest-scoring documents
        top_10_indices = torch.argsort(scores, descending=True)[:10].cpu().numpy()
        
        # Select the top 10 document IDs based on the indices
        top_10_doc_ids = np.array(top_100_doc_ids)[top_10_indices].tolist()
        
        # Store the result for this query (query_id starts from 0 to 1999)
        final_results[query_id] = top_10_doc_ids

    return final_results

# Example usage
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load the precomputed document embeddings from "doc_embedding.pkl"
    doc_embeddings = load_precomputed_embeddings("/kaggle/input/dis-embedding/doc_embedding.pkl")
    
    # Load the trained ranking model from the given path
    model = load_model("/kaggle/input/listwise/ranking_model_50.pth", device)

    # Load the top 100 candidates (only doc IDs) from the CSV file
    candidates_df = load_top_100_candidates("/kaggle/input/top200-cosine/top_candidates.csv")

    # Rank the top 100 documents and select top 10 using the trained model
    final_results = rank_top_10_for_each_query(model, candidates_df, doc_embeddings, device)

    # Store the final results in a CSV file with "id" and "docids" columns
    with open('/kaggle/working/top_10_results.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "docids"])  # Column names
        
        for query_id, doc_ids in final_results.items():
            writer.writerow([query_id, json.dumps(doc_ids)])  # Save query_id and doc_ids as JSON list

    print("Top 10 document results stored in 'query_top_10_results.csv'")


/tmp/ipykernel_31/1201425435.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
/tmp/ipykernel_31/120142

Top 10 document results stored in 'query_top_10_results.csv'
